In [1]:
import torch
import torch.nn.functional as F  
from torch import optim 
from torch import nn
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import torch.nn.init as init
import random
import time

### Class for the Neural Network

In [2]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes, hidden_layer_sizes, activation_function, apply_softmax = False):
        super(NN, self).__init__()

        seed = 18
        torch.manual_seed(seed)

        self.activation = activation_function
        self.apply_softmax = apply_softmax

        # Input layer
        self.input_layer = nn.Linear(input_size, hidden_layer_sizes[0])
        init.kaiming_uniform_(self.input_layer.weight, mode='fan_in', nonlinearity=activation_function.__name__)


        # Hidden layers
        self.hidden_layers = nn.ModuleList([
            nn.Linear(hidden_layer_sizes[i], hidden_layer_sizes[i + 1])
            for i in range(len(hidden_layer_sizes) - 1)
        ])

        for layer in self.hidden_layers:
            init.kaiming_uniform_(layer.weight, mode='fan_in', nonlinearity=activation_function.__name__)


        # Output layer
        self.output_layer = nn.Linear(hidden_layer_sizes[-1], num_classes)
        init.kaiming_uniform_(self.output_layer.weight, mode='fan_in', nonlinearity=activation_function.__name__)


    def forward(self, x):
        x = x.float()
        x = self.activation(self.input_layer(x))

        # Process through hidden layers
        for layer in self.hidden_layers:
            x = self.activation(layer(x))

        if self.apply_softmax:
            x = F.softmax(self.output_layer(x), dim=1)
        else:
            x = self.output_layer(x)

        return x
    

### Function to train the model

In [3]:
def train_model(model, train_loader, optimizer, criterion, num_epochs):
    for epoch in range(num_epochs):
        total_loss = 0.0  # Initialize total loss for the epoch
        num_batches = len(train_loader)

        for batch_idx, (data, targets) in enumerate(train_loader):
            data = data.reshape(data.shape[0], -1)

            scores = model(data)
            loss = criterion(scores, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()  # Accumulate the batch loss

        average_loss = total_loss / num_batches
        print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {average_loss}")
    

### Function to calculate the accuracy

In [4]:
def check_accuracy(loader, model):

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:

            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples


### Function to split the data into train and test

In [5]:
def train_test_split(data: pd.DataFrame, target_label : str, test_size=0.2, return_torch=None):
        
    # split the data into train and test
    train = data.sample(frac=(1-test_size),random_state=200)
    test = data.drop(train.index)
    
    # split the train and test into X and Y
    train_X = train.drop([target_label], axis=1).values
    train_Y = train[target_label].values
    test_X = test.drop([target_label], axis=1).values
    test_Y = test[target_label].values
    
    if return_torch:
        train_X = torch.tensor(train_X)
        train_Y = torch.tensor(train_Y)
        test_X = torch.tensor(test_X)
        test_Y = torch.tensor(test_Y)
    
    return train_X, train_Y, test_X, test_Y

## Function for Grid Search

In [6]:
def grid_search(hidden_layer_sizes_list, activation_functions, learning_rates, batch_sizes, num_epochs_list, train_loader, test_loader):
    best_accuracy = 0.0
    best_combination = None
    results = []

    for hidden_layer_sizes in hidden_layer_sizes_list:
        for activation_function in activation_functions:
            for learning_rate in learning_rates:
                for batch_size in batch_sizes:
                    for num_epochs in num_epochs_list:
                
                        model = NN(input_size=train_loader.dataset.tensors[0].shape[1],
                                   num_classes=num_classes,
                                   hidden_layer_sizes=hidden_layer_sizes,
                                   activation_function=activation_function)
                        criterion = nn.CrossEntropyLoss()
                        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

                        train_start_time = time.time()
                        train_model(model, train_loader, optimizer, criterion, num_epochs)
                        train_end_time = time.time()
                        
                        accuracy_train = check_accuracy(train_loader, model)
                        accuracy_test = check_accuracy(test_loader, model)

                        result = {
                            'Hidden Layer Sizes': hidden_layer_sizes,
                            'Activation Function': activation_function.__name__,
                            'Learning Rate': learning_rate,
                            'Batch Size': batch_size,
                            'Number of Epochs': num_epochs,
                            'Accuracy (Train)': accuracy_train.item(),
                            'Accuracy (Test)': accuracy_test.item(),
                            'Training Time': train_end_time - train_start_time
                        }

                        results.append(result)
                        
                        if accuracy_test > best_accuracy:
                            best_accuracy = accuracy_test
                            best_combination = result

    results_df = pd.DataFrame(results)
    return results_df, best_accuracy, best_combination

## Function for Random Search

In [7]:
def random_search(num_iterations, initial_configuration, param_ranges, train_loader, test_loader):
    best_accuracy = 0.0
    best_combination = initial_configuration
    train_times = []
    results = []

    for _ in range(num_iterations):
        # Randomly sample hyperparameter values inthe specified ranges
        sampled_configuration = {
            'Hidden Layer Sizes': [random.randint(param_ranges['min_hidden'], param_ranges['max_hidden']) for _ in range(random.randint(param_ranges['min_layers'], param_ranges['max_layers']))],
            'Activation Function': random.choice(param_ranges['activation_functions']),
            'Learning Rate': random.uniform(param_ranges['min_lr'], param_ranges['max_lr']),
            'Batch Size': random.choice(param_ranges['batch_sizes']),
            'Number of Epochs': random.choice(param_ranges['num_epochs'])
        }

        train_start_time = time.time()
        model = NN(input_size=train_loader.dataset.tensors[0].shape[1],
                   num_classes=num_classes,
                   hidden_layer_sizes=sampled_configuration['Hidden Layer Sizes'],
                   activation_function=sampled_configuration['Activation Function'])
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=sampled_configuration['Learning Rate'])

        train_model(model, train_loader, optimizer, criterion, sampled_configuration['Number of Epochs'])

        train_end_time = time.time()
        train_times.append(train_end_time - train_start_time)
        accuracy_test = check_accuracy(test_loader, model)

        result_entry = {
            'Hidden Layer Sizes': sampled_configuration['Hidden Layer Sizes'],
            'Activation Function': sampled_configuration['Activation Function'].__name__,
            'Learning Rate': sampled_configuration['Learning Rate'],
            'Batch Size': sampled_configuration['Batch Size'],
            'Number of Epochs': sampled_configuration['Number of Epochs'],
            'Accuracy': accuracy_test.item(),
            'Training Time': train_end_time - train_start_time
        }

        results.append(result_entry)

        if accuracy_test > best_accuracy:
            best_accuracy = accuracy_test
            best_combination = sampled_configuration

    results_df = pd.DataFrame(results)
    return best_combination, best_accuracy, results_df

## Function for Local Search

In [8]:
def local_search(num_iterations, initial_configuration, param_ranges, train_loader, test_loader):
    best_accuracy = 0.0
    best_combination = None
    current_configuration = initial_configuration
    train_times = []
    results = []

    for _ in range(num_iterations):
        # small changes to the current configuration
        new_configuration = {
            'Hidden Layer Sizes': [
                max(1, size + random.randint(-1, 1)) for size in current_configuration['Hidden Layer Sizes']
            ],
            'Activation Function': random.choice(param_ranges['activation_functions']),
            'Learning Rate': max(param_ranges['min_lr'], min(param_ranges['max_lr'], current_configuration['Learning Rate'] + random.uniform(-0.01, 0.01))),
            'Batch Size': random.choice(param_ranges['batch_sizes']),
            'Number of Epochs': max(1, current_configuration['Number of Epochs'] + random.randint(-1, 1))
        }

        train_start_time = time.time()
        model = NN(input_size=train_loader.dataset.tensors[0].shape[1],
                   num_classes=num_classes,
                   hidden_layer_sizes=new_configuration['Hidden Layer Sizes'],
                   activation_function=new_configuration['Activation Function'])
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=new_configuration['Learning Rate'])

        train_model(model, train_loader, optimizer, criterion, new_configuration['Number of Epochs'])

        train_end_time = time.time()
        train_times.append(train_end_time - train_start_time)
        accuracy_test = check_accuracy(test_loader, model)

        result_entry = {
            'Hidden Layer Sizes': new_configuration['Hidden Layer Sizes'],
            'Activation Function': new_configuration['Activation Function'].__name__,
            'Learning Rate': new_configuration['Learning Rate'],
            'Batch Size': new_configuration['Batch Size'],
            'Number of Epochs': new_configuration['Number of Epochs'],
            'Accuracy': accuracy_test.item(),
            'Training Time': train_end_time - train_start_time
        }

        results.append(result_entry)

        if accuracy_test > best_accuracy:
            best_accuracy = accuracy_test
            best_combination = new_configuration

        # Update the current configuration for the next iteration
        current_configuration = new_configuration

    results_df = pd.DataFrame(results)
    return best_combination, best_accuracy, results_df

## Testing the model on the wine quality dataset

#### Loading the dataset

In [9]:
wine_quality = pd.read_csv('./preprocessed-datasets/wine_quality_prepro.csv', index_col=0)
wine_quality.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,class,wine_type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


#### Splitting the dataset into training and testing sets, converting to PyTorch tensors and creating PyTorch DataLoaders

In [10]:
train_X, train_Y, test_X, test_Y = train_test_split(wine_quality, "class", return_torch=True)

dataset = TensorDataset(train_X, train_Y)
train_loader = DataLoader(dataset, batch_size=32, shuffle=False)

dataset = TensorDataset(test_X, test_Y)
test_loader = DataLoader(dataset, batch_size=32, shuffle=False)


#### Creating the model, training and testing

In [11]:
input_size = train_X.shape[1] # number of features in wine quality dataset
num_classes = 10 # 10 classes in wine quality dataset
learning_rate = 0.01
batch_size = 64
num_epochs = 10
hidden_layer_sizes = [25,30]
activation_function = F.tanh

model = NN(input_size=train_X.shape[1], num_classes=num_classes, hidden_layer_sizes=hidden_layer_sizes, activation_function=activation_function)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_model(model, train_loader, optimizer, criterion, num_epochs)

print(f"Accuracy on training set: {check_accuracy(train_loader, model)}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)}")

Epoch 1/10, Average Loss: 1.3338039038371454
Epoch 2/10, Average Loss: 1.2913920031003425
Epoch 3/10, Average Loss: 1.2882495277498398
Epoch 4/10, Average Loss: 1.287879331711611
Epoch 5/10, Average Loss: 1.2764818719559652
Epoch 6/10, Average Loss: 1.273354259736699
Epoch 7/10, Average Loss: 1.2613155139735872
Epoch 8/10, Average Loss: 1.259151681069216
Epoch 9/10, Average Loss: 1.2574602010791287
Epoch 10/10, Average Loss: 1.2471537063458213
Accuracy on training set: 0.44132357835769653
Accuracy on test set: 0.4470588266849518


#### Testing out Grid Search on the wine quality dataset

In [12]:
hidden_layer_sizes_list = [[25, 30], [20, 25, 30]]
activation_functions = [F.tanh, F.relu]
learning_rates = [0.01, 0.001]
batch_sizes = [32, 64]
num_epochs_list = [10, 15]

grid_results_df_wine, best_accuracy_wine, best_combination_wine = grid_search(hidden_layer_sizes_list, activation_functions, learning_rates, batch_sizes, num_epochs_list, train_loader, test_loader)

print("Best Accuracy:", best_accuracy_wine)
print("Best Combination:")
print(best_combination_wine)

# Saving results
#results_df_wine.to_csv('grid_search_results.csv', index=False)

Epoch 1/10, Average Loss: 1.3338039038371454
Epoch 2/10, Average Loss: 1.2913920031003425
Epoch 3/10, Average Loss: 1.2882495277498398
Epoch 4/10, Average Loss: 1.287879331711611
Epoch 5/10, Average Loss: 1.2764818719559652
Epoch 6/10, Average Loss: 1.273354259736699
Epoch 7/10, Average Loss: 1.2613155139735872
Epoch 8/10, Average Loss: 1.259151681069216
Epoch 9/10, Average Loss: 1.2574602010791287
Epoch 10/10, Average Loss: 1.2471537063458213
Epoch 1/15, Average Loss: 1.3338039038371454
Epoch 2/15, Average Loss: 1.2913920031003425
Epoch 3/15, Average Loss: 1.2882495277498398
Epoch 4/15, Average Loss: 1.287879331711611
Epoch 5/15, Average Loss: 1.2764818719559652
Epoch 6/15, Average Loss: 1.273354259736699
Epoch 7/15, Average Loss: 1.2613155139735872
Epoch 8/15, Average Loss: 1.259151681069216
Epoch 9/15, Average Loss: 1.2574602010791287
Epoch 10/15, Average Loss: 1.2471537063458213
Epoch 11/15, Average Loss: 1.2420796514288779
Epoch 12/15, Average Loss: 1.2381280730107078
Epoch 13/15,

Epoch 9/10, Average Loss: 1.2536153712886975
Epoch 10/10, Average Loss: 1.247289777899081
Epoch 1/15, Average Loss: 9.377237459633248
Epoch 2/15, Average Loss: 1.4767584391166828
Epoch 3/15, Average Loss: 1.3564692805149803
Epoch 4/15, Average Loss: 1.3102094132476059
Epoch 5/15, Average Loss: 1.2892052538555825
Epoch 6/15, Average Loss: 1.2768256934873896
Epoch 7/15, Average Loss: 1.2672780251210451
Epoch 8/15, Average Loss: 1.2594708646733337
Epoch 9/15, Average Loss: 1.2536153712886975
Epoch 10/15, Average Loss: 1.247289777899081
Epoch 11/15, Average Loss: 1.2416112975108844
Epoch 12/15, Average Loss: 1.2359264591720207
Epoch 13/15, Average Loss: 1.2301301670952078
Epoch 14/15, Average Loss: 1.224707898918105
Epoch 15/15, Average Loss: 1.2197116788910942
Epoch 1/10, Average Loss: 1.3134170412285928
Epoch 2/10, Average Loss: 1.2745691197781475
Epoch 3/10, Average Loss: 1.2602842044245246
Epoch 4/10, Average Loss: 1.255159665836147
Epoch 5/10, Average Loss: 1.2602674778253755
Epoch 6/

Epoch 6/15, Average Loss: 1.4147324657147646
Epoch 7/15, Average Loss: 1.3891667884551675
Epoch 8/15, Average Loss: 1.3638630611764873
Epoch 9/15, Average Loss: 1.345775969189369
Epoch 10/15, Average Loss: 1.3330556205445272
Epoch 11/15, Average Loss: 1.320689352377792
Epoch 12/15, Average Loss: 1.3102229701960746
Epoch 13/15, Average Loss: 1.302969576756647
Epoch 14/15, Average Loss: 1.29046348998883
Epoch 15/15, Average Loss: 1.2835565172821466
Epoch 1/10, Average Loss: 8.816645726835802
Epoch 2/10, Average Loss: 1.8097324327457172
Epoch 3/10, Average Loss: 1.591396429056039
Epoch 4/10, Average Loss: 1.5077126728245085
Epoch 5/10, Average Loss: 1.4583211479011489
Epoch 6/10, Average Loss: 1.4147324657147646
Epoch 7/10, Average Loss: 1.3891667884551675
Epoch 8/10, Average Loss: 1.3638630611764873
Epoch 9/10, Average Loss: 1.345775969189369
Epoch 10/10, Average Loss: 1.3330556205445272
Epoch 1/15, Average Loss: 8.816645726835802
Epoch 2/15, Average Loss: 1.8097324327457172
Epoch 3/15, 

In [13]:
grid_results_df_wine['dataset'] = 'wine_quality'
grid_results_df_wine

,Hidden Layer Sizes,Activation Function,Learning Rate,Batch Size,Number of Epochs,Accuracy (Train),Accuracy (Test),Training Time,dataset
0,"[25, 30]",tanh,0.010,32,10,0.441324,0.447059,1.888755,wine_quality
1,"[25, 30]",tanh,0.010,32,15,0.443247,0.462745,3.049353,wine_quality
2,"[25, 30]",tanh,0.010,64,10,0.441324,0.447059,1.875426,wine_quality
3,"[25, 30]",tanh,0.010,64,15,0.443247,0.462745,2.630327,wine_quality
4,"[25, 30]",tanh,0.001,32,10,0.449211,0.453595,1.687192,wine_quality
5,"[25, 30]",tanh,0.001,32,15,0.446518,0.454902,2.637031,wine_quality
6,"[25, 30]",tanh,0.001,64,10,0.449211,0.453595,1.803701,wine_quality
7,"[25, 30]",tanh,0.001,64,15,0.446518,0.454902,2.624947,wine_quality
8,"[25, 30]",relu,0.010,32,10,0.503848,0.475817,2.112982,wine_quality
9,"[25, 30]",relu,0.010,32,15,0.497499,0.484967,3.312419,wine_quality


#### Testing out random search on the wine quality dataset

In [14]:
num_iterations = 10
param_ranges = {
    'min_hidden': 5,
    'max_hidden': 50,
    'min_layers': 1,
    'max_layers': 3,
    'activation_functions': [F.relu, F.tanh, F.sigmoid],
    'min_lr': 0.001,
    'max_lr': 0.1,
    'batch_sizes': [32, 64, 128],
    'num_epochs': [5, 10, 15]
}

initial_configuration = {
    'Hidden Layer Sizes': [25],
    'Activation Function': F.relu,
    'Learning Rate': 0.01,
    'Batch Size': 64,
    'Number of Epochs': 10
}

best_config, best_accuracy, random_results_table_wine = random_search(num_iterations, initial_configuration, param_ranges, train_loader, test_loader)
print("Best Configuration:", best_config)
print("Best Accuracy:", best_accuracy)


Epoch 1/10, Average Loss: 1.4267507520921392
Epoch 2/10, Average Loss: 1.3992398114292168
Epoch 3/10, Average Loss: 1.3986807257119864
Epoch 4/10, Average Loss: 1.3990603743886656
Epoch 5/10, Average Loss: 1.3991614748363845
Epoch 6/10, Average Loss: 1.399175955474011
Epoch 7/10, Average Loss: 1.3990336693137702
Epoch 8/10, Average Loss: 1.399297660845189
Epoch 9/10, Average Loss: 1.399294968763012
Epoch 10/10, Average Loss: 1.3992904256457932
Epoch 1/15, Average Loss: 3.26213581284131
Epoch 2/15, Average Loss: 1.5386129248361646
Epoch 3/15, Average Loss: 1.3514377745382624
Epoch 4/15, Average Loss: 1.2727943241961894
Epoch 5/15, Average Loss: 1.2372236471234654
Epoch 6/15, Average Loss: 1.212742452606833
Epoch 7/15, Average Loss: 1.1895369631381123
Epoch 8/15, Average Loss: 1.1814518841497736
Epoch 9/15, Average Loss: 1.1680733971069195
Epoch 10/15, Average Loss: 1.157967927631425
Epoch 11/15, Average Loss: 1.1476052224270405
Epoch 12/15, Average Loss: 1.15077039581135
Epoch 13/15, Av

In [15]:
random_results_table_wine['dataset'] = 'wine_quality'
random_results_table_wine

,Hidden Layer Sizes,Activation Function,Learning Rate,Batch Size,Number of Epochs,Accuracy,Training Time,dataset
0,"[36, 34, 26]",tanh,0.071628,128,10,0.284967,2.066317,wine_quality
1,"[46, 37]",relu,0.008342,32,15,0.481046,2.595385,wine_quality
2,"[16, 23, 40]",relu,0.063862,64,5,0.458824,0.945579,wine_quality
3,"[41, 20]",sigmoid,0.010363,64,10,0.486275,1.682182,wine_quality
4,"[41, 37, 41]",tanh,0.073547,32,15,0.284967,2.961220,wine_quality
5,"[13, 30, 44]",tanh,0.058814,64,5,0.284967,0.952416,wine_quality
6,[48],sigmoid,0.046612,32,5,0.458824,0.739777,wine_quality
7,[20],relu,0.049819,64,15,0.473203,2.199781,wine_quality
8,"[19, 49]",sigmoid,0.019329,64,10,0.448366,1.675180,wine_quality
9,"[7, 50]",sigmoid,0.074720,64,15,0.284967,2.509224,wine_quality


In [16]:
#### Testing out Local Search on wine quality dataset

In [17]:
num_iterations = 10
param_ranges = {
    'min_hidden': 5,
    'max_hidden': 50,
    'min_layers': 1,
    'max_layers': 3,
    'activation_functions': [F.relu, F.tanh, F.sigmoid],
    'min_lr': 0.001,
    'max_lr': 0.1,
    'batch_sizes': [32, 64, 128],
    'num_epochs': [5, 10, 15]
}

initial_configuration = {
    'Hidden Layer Sizes': [25],
    'Activation Function': F.relu,
    'Learning Rate': 0.01,
    'Batch Size': 64,
    'Number of Epochs': 10
}

best_config, best_accuracy, local_results_table_wine = local_search(num_iterations, initial_configuration, param_ranges, train_loader, test_loader)
print("Best Configuration:", best_config)
print("Best Accuracy:", best_accuracy)

Epoch 1/11, Average Loss: 1.3584940192158237
Epoch 2/11, Average Loss: 1.2512979511103015
Epoch 3/11, Average Loss: 1.2212291678036649
Epoch 4/11, Average Loss: 1.2004704731373699
Epoch 5/11, Average Loss: 1.1875284727365694
Epoch 6/11, Average Loss: 1.170317331706088
Epoch 7/11, Average Loss: 1.1579291279330575
Epoch 8/11, Average Loss: 1.1476109616595542
Epoch 9/11, Average Loss: 1.1442622260813333
Epoch 10/11, Average Loss: 1.1359261252397408
Epoch 11/11, Average Loss: 1.132720259069665
Epoch 1/10, Average Loss: 1.3082550573934075
Epoch 2/10, Average Loss: 1.2724334762140286
Epoch 3/10, Average Loss: 1.2610128562143244
Epoch 4/10, Average Loss: 1.2563806969695297
Epoch 5/10, Average Loss: 1.251952894260547
Epoch 6/10, Average Loss: 1.2427995914330512
Epoch 7/10, Average Loss: 1.2269865910699762
Epoch 8/10, Average Loss: 1.203735042569096
Epoch 9/10, Average Loss: 1.1786008009149984
Epoch 10/10, Average Loss: 1.1722392670216004
Epoch 1/11, Average Loss: 1.3410476277942307
Epoch 2/11,

In [18]:
local_results_table_wine

,Hidden Layer Sizes,Activation Function,Learning Rate,Batch Size,Number of Epochs,Accuracy,Training Time
0,[26],sigmoid,0.008324,32,11,0.494118,2.548559
1,[25],sigmoid,0.012688,32,10,0.462745,4.352979
2,[25],tanh,0.015207,32,11,0.454902,2.614930
3,[25],sigmoid,0.020691,64,12,0.467974,1.869798
4,[26],tanh,0.023087,128,11,0.284967,4.247971
5,[26],sigmoid,0.024981,32,10,0.471895,1.592727
6,[26],sigmoid,0.017405,64,9,0.456209,1.362743
7,[27],tanh,0.024421,32,10,0.286275,2.294606
8,[26],relu,0.030700,32,10,0.484967,1.626590
9,[26],sigmoid,0.023405,32,10,0.475817,1.640130


## Testing the model on the congressional voting dataset

#### Loading the dataset

In [19]:
cong_voting = pd.read_csv('./preprocessed-datasets/CongressionVoting_prepro.csv')
# encode class value democrat as 1 and republican as 0
cong_voting['class'] = cong_voting['class'].map({'democrat': 1, 'republican': 0})
cong_voting.head()

,ID,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-crporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa,class
0,140,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1
1,383,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1
2,201,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1
3,297,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0
4,309,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0


#### Splitting the dataset into training and testing sets, converting to PyTorch tensors and creating PyTorch DataLoaders

In [20]:
train_X, train_Y, test_X, test_Y = train_test_split(cong_voting, "class", return_torch=True)

dataset = TensorDataset(train_X, train_Y)
train_loader = DataLoader(dataset, batch_size=32, shuffle=False)

dataset = TensorDataset(test_X, test_Y)
test_loader = DataLoader(dataset, batch_size=32, shuffle=False)

#### Creating the model, training and testing

In [21]:
input_size = train_X.shape[1] # number of features in congr voting dataset
num_classes = 2 # 2 classes in congr voting dataset
learning_rate = 0.01
batch_size = 64
num_epochs = 10
hidden_layer_sizes = [25,30]
activation_function = F.tanh

model = NN(input_size=train_X.shape[1], num_classes=num_classes, hidden_layer_sizes=hidden_layer_sizes, activation_function=activation_function)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_model(model, train_loader, optimizer, criterion, num_epochs)

print(f"Accuracy on training set: {check_accuracy(train_loader, model)}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)}")

Epoch 1/10, Average Loss: 0.8204039831956228
Epoch 2/10, Average Loss: 0.6987561782201132
Epoch 3/10, Average Loss: 0.6630964974562327
Epoch 4/10, Average Loss: 0.5736258924007416
Epoch 5/10, Average Loss: 0.4424207905928294
Epoch 6/10, Average Loss: 0.3960420439640681
Epoch 7/10, Average Loss: 0.29747725029786426
Epoch 8/10, Average Loss: 0.2279346063733101
Epoch 9/10, Average Loss: 0.22333091124892235
Epoch 10/10, Average Loss: 0.3738237793246905
Accuracy on training set: 0.959770143032074
Accuracy on test set: 0.8604651093482971


## Testing the model on bank marketing dataset

#### Loading and preparing dataset

In [22]:
bank_marketing = pd.read_csv('./preprocessed-datasets/bank_marketing_prepro.csv')

In [23]:
column_to_move = 'class'

# Move class to the last index
columns = [col for col in bank_marketing.columns if col != column_to_move] + [column_to_move]
bank_marketing = bank_marketing[columns]

bank_marketing.drop('Unnamed: 0', axis=1,inplace=True)

bank_marketing.head()

,age,default,housing,loan,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,...,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,poutcome_failure,poutcome_nonexistent,poutcome_success,class
0,56,0.0,0.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,0,0,0,0,0,0,1,0,0
1,57,0.0,0.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,1,0,0,0,0,0,1,0,0
2,37,0.0,1.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,1,0,0,0,0,0,1,0,0
3,40,0.0,0.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,0,0,0,0,0,0,1,0,0
4,56,0.0,0.0,1.0,1,999,0,1.1,93.994,-36.4,...,0,1,0,0,0,0,0,1,0,0


In [24]:
bank_marketing.columns

Index(['age', 'default', 'housing', 'loan', 'campaign', 'pdays', 'previous',
       'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m',
       'nr.employed', 'job_blue-collar', 'job_management', 'job_other',
       'job_self-employed', 'job_serivces', 'job_technician',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'poutcome_failure', 'poutcome_nonexistent',
       'poutcome_success', 'class'],
      dtype='object')

#### Splitting the dataset into training and testing sets, converting to PyTorch tensors and creating PyTorch DataLoaders

In [25]:
train_X, train_Y, test_X, test_Y = train_test_split(bank_marketing, "class", return_torch=True)

dataset = TensorDataset(train_X, train_Y)
train_loader = DataLoader(dataset, batch_size=32, shuffle=False)

dataset = TensorDataset(test_X, test_Y)
test_loader = DataLoader(dataset, batch_size=32, shuffle=False)

#### Creating the model, training and testing

In [26]:
input_size = train_X.shape[1] # number of features in congr voting dataset
num_classes = 2 # 2 classes in congr voting dataset
learning_rate = 0.01
batch_size = 64
num_epochs = 10
hidden_layer_sizes = [25,30]
activation_function = F.tanh

model = NN(input_size=train_X.shape[1], num_classes=num_classes, hidden_layer_sizes=hidden_layer_sizes, activation_function=activation_function)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_model(model, train_loader, optimizer, criterion, num_epochs)

print(f"Accuracy on training set: {check_accuracy(train_loader, model)}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)}")

Epoch 1/10, Average Loss: 0.33090905211142546
Epoch 2/10, Average Loss: 0.32768635471686
Epoch 3/10, Average Loss: 0.327213676422111
Epoch 4/10, Average Loss: 0.3280542928255299
Epoch 5/10, Average Loss: 0.32849315527139356
Epoch 6/10, Average Loss: 0.32820502676937763
Epoch 7/10, Average Loss: 0.3284298519956545
Epoch 8/10, Average Loss: 0.3284010060674068
Epoch 9/10, Average Loss: 0.3283500720249507
Epoch 10/10, Average Loss: 0.3283919007888118
Accuracy on training set: 0.8986949920654297
Accuracy on test set: 0.8928138017654419


# Test Grid search over all three datasets

In [27]:
datasets = {'wine_quality': wine_quality, 'cong_voting': cong_voting, 'bank_marketing': bank_marketing} #
hidden_layer_sizes_list = [[25, 30], [20, 25, 30]]
activation_functions = [F.tanh, F.relu] 
learning_rates = [0.01, 0.001]
batch_sizes = [32, 64]
num_epochs_list = [10, 15]
all_results = []

for dataset_name, dataset in datasets.items():
    train_X, train_Y, test_X, test_Y = train_test_split(dataset, "class", return_torch=True)

    train_data = TensorDataset(train_X, train_Y)
    train_loader = DataLoader(train_data, batch_size=32, shuffle=False)

    test_data = TensorDataset(test_X, test_Y)
    test_loader = DataLoader(test_data, batch_size=32, shuffle=False)
    
    input_size = train_X.shape[1]

    if dataset_name == 'wine_quality':
        num_classes = 10
    else:
        num_classes = len(np.unique(train_Y))
    #print(len(np.unique(train_Y)))

    grid_results, best_accuracy, best_combination = grid_search(
        hidden_layer_sizes_list, activation_functions, learning_rates, batch_sizes, num_epochs_list, train_loader, test_loader)

    grid_results['dataset'] = dataset_name
    
    all_results.append(grid_results)

grid_results_df = pd.concat(all_results, ignore_index=True)

Epoch 1/10, Average Loss: 1.3338039038371454
Epoch 2/10, Average Loss: 1.2913920031003425
Epoch 3/10, Average Loss: 1.2882495277498398
Epoch 4/10, Average Loss: 1.287879331711611
Epoch 5/10, Average Loss: 1.2764818719559652
Epoch 6/10, Average Loss: 1.273354259736699
Epoch 7/10, Average Loss: 1.2613155139735872
Epoch 8/10, Average Loss: 1.259151681069216
Epoch 9/10, Average Loss: 1.2574602010791287
Epoch 10/10, Average Loss: 1.2471537063458213
Epoch 1/15, Average Loss: 1.3338039038371454
Epoch 2/15, Average Loss: 1.2913920031003425
Epoch 3/15, Average Loss: 1.2882495277498398
Epoch 4/15, Average Loss: 1.287879331711611
Epoch 5/15, Average Loss: 1.2764818719559652
Epoch 6/15, Average Loss: 1.273354259736699
Epoch 7/15, Average Loss: 1.2613155139735872
Epoch 8/15, Average Loss: 1.259151681069216
Epoch 9/15, Average Loss: 1.2574602010791287
Epoch 10/15, Average Loss: 1.2471537063458213
Epoch 11/15, Average Loss: 1.2420796514288779
Epoch 12/15, Average Loss: 1.2381280730107078
Epoch 13/15,

Epoch 9/10, Average Loss: 1.2536153712886975
Epoch 10/10, Average Loss: 1.247289777899081
Epoch 1/15, Average Loss: 9.377237459633248
Epoch 2/15, Average Loss: 1.4767584391166828
Epoch 3/15, Average Loss: 1.3564692805149803
Epoch 4/15, Average Loss: 1.3102094132476059
Epoch 5/15, Average Loss: 1.2892052538555825
Epoch 6/15, Average Loss: 1.2768256934873896
Epoch 7/15, Average Loss: 1.2672780251210451
Epoch 8/15, Average Loss: 1.2594708646733337
Epoch 9/15, Average Loss: 1.2536153712886975
Epoch 10/15, Average Loss: 1.247289777899081
Epoch 11/15, Average Loss: 1.2416112975108844
Epoch 12/15, Average Loss: 1.2359264591720207
Epoch 13/15, Average Loss: 1.2301301670952078
Epoch 14/15, Average Loss: 1.224707898918105
Epoch 15/15, Average Loss: 1.2197116788910942
Epoch 1/10, Average Loss: 1.3134170412285928
Epoch 2/10, Average Loss: 1.2745691197781475
Epoch 3/10, Average Loss: 1.2602842044245246
Epoch 4/10, Average Loss: 1.255159665836147
Epoch 5/10, Average Loss: 1.2602674778253755
Epoch 6/

Epoch 6/15, Average Loss: 1.4147324657147646
Epoch 7/15, Average Loss: 1.3891667884551675
Epoch 8/15, Average Loss: 1.3638630611764873
Epoch 9/15, Average Loss: 1.345775969189369
Epoch 10/15, Average Loss: 1.3330556205445272
Epoch 11/15, Average Loss: 1.320689352377792
Epoch 12/15, Average Loss: 1.3102229701960746
Epoch 13/15, Average Loss: 1.302969576756647
Epoch 14/15, Average Loss: 1.29046348998883
Epoch 15/15, Average Loss: 1.2835565172821466
Epoch 1/10, Average Loss: 8.816645726835802
Epoch 2/10, Average Loss: 1.8097324327457172
Epoch 3/10, Average Loss: 1.591396429056039
Epoch 4/10, Average Loss: 1.5077126728245085
Epoch 5/10, Average Loss: 1.4583211479011489
Epoch 6/10, Average Loss: 1.4147324657147646
Epoch 7/10, Average Loss: 1.3891667884551675
Epoch 8/10, Average Loss: 1.3638630611764873
Epoch 9/10, Average Loss: 1.345775969189369
Epoch 10/10, Average Loss: 1.3330556205445272
Epoch 1/15, Average Loss: 8.816645726835802
Epoch 2/15, Average Loss: 1.8097324327457172
Epoch 3/15, 

Epoch 2/15, Average Loss: 13.70142294963201
Epoch 3/15, Average Loss: 7.520770311355591
Epoch 4/15, Average Loss: 8.185037533442179
Epoch 5/15, Average Loss: 2.8259220321973166
Epoch 6/15, Average Loss: 2.318928783138593
Epoch 7/15, Average Loss: 1.678759495417277
Epoch 8/15, Average Loss: 0.986444835861524
Epoch 9/15, Average Loss: 0.6860190977652868
Epoch 10/15, Average Loss: 0.522232914964358
Epoch 11/15, Average Loss: 0.37667860339085263
Epoch 12/15, Average Loss: 0.30535699675480527
Epoch 13/15, Average Loss: 0.3000682443380356
Epoch 14/15, Average Loss: 0.24604910363753638
Epoch 15/15, Average Loss: 0.2401899273196856
Epoch 1/10, Average Loss: 52.3094285329183
Epoch 2/10, Average Loss: 13.70142294963201
Epoch 3/10, Average Loss: 7.520770311355591
Epoch 4/10, Average Loss: 8.185037533442179
Epoch 5/10, Average Loss: 2.8259220321973166
Epoch 6/10, Average Loss: 2.318928783138593
Epoch 7/10, Average Loss: 1.678759495417277
Epoch 8/10, Average Loss: 0.986444835861524
Epoch 9/10, Aver

Epoch 10/10, Average Loss: 0.528020516037941
Epoch 1/15, Average Loss: 0.8128066658973694
Epoch 2/15, Average Loss: 0.6831894516944885
Epoch 3/15, Average Loss: 0.6026290853818258
Epoch 4/15, Average Loss: 0.6084774732589722
Epoch 5/15, Average Loss: 0.581465482711792
Epoch 6/15, Average Loss: 0.5671104590098063
Epoch 7/15, Average Loss: 0.564067671696345
Epoch 8/15, Average Loss: 0.5438016951084137
Epoch 9/15, Average Loss: 0.5356756647427877
Epoch 10/15, Average Loss: 0.528020516037941
Epoch 11/15, Average Loss: 0.5085686445236206
Epoch 12/15, Average Loss: 0.4716069499651591
Epoch 13/15, Average Loss: 0.46406584481398266
Epoch 14/15, Average Loss: 0.45526200036207837
Epoch 15/15, Average Loss: 0.43859848380088806
Epoch 1/10, Average Loss: 0.8128066658973694
Epoch 2/10, Average Loss: 0.6831894516944885
Epoch 3/10, Average Loss: 0.6026290853818258
Epoch 4/10, Average Loss: 0.6084774732589722
Epoch 5/10, Average Loss: 0.581465482711792
Epoch 6/10, Average Loss: 0.5671104590098063
Epoch

Epoch 5/15, Average Loss: 0.32554703892900344
Epoch 6/15, Average Loss: 0.31441083868911257
Epoch 7/15, Average Loss: 0.31894920878112315
Epoch 8/15, Average Loss: 0.32183119829707935
Epoch 9/15, Average Loss: 0.33429822765507744
Epoch 10/15, Average Loss: 0.3506328809102183
Epoch 11/15, Average Loss: 0.3506433578515516
Epoch 12/15, Average Loss: 0.35055537434312906
Epoch 13/15, Average Loss: 0.350455629623052
Epoch 14/15, Average Loss: 0.35036220992508443
Epoch 15/15, Average Loss: 0.35030375881825837
Epoch 1/10, Average Loss: 5.5405291947448685
Epoch 2/10, Average Loss: 2.752882260165062
Epoch 3/10, Average Loss: 2.1771973654129253
Epoch 4/10, Average Loss: 1.8716167330812523
Epoch 5/10, Average Loss: 2.422839765288852
Epoch 6/10, Average Loss: 1.8085828310284378
Epoch 7/10, Average Loss: 1.98093881867435
Epoch 8/10, Average Loss: 1.9040547810870319
Epoch 9/10, Average Loss: 1.9979055106412462
Epoch 10/10, Average Loss: 1.7279514920587051
Epoch 1/15, Average Loss: 5.5405291947448685


Epoch 11/15, Average Loss: 0.3503446742747594
Epoch 12/15, Average Loss: 0.3502924434551336
Epoch 13/15, Average Loss: 0.3502500360596527
Epoch 14/15, Average Loss: 0.350221598893404
Epoch 15/15, Average Loss: 0.3502073515098072
Epoch 1/10, Average Loss: 4.014158828241413
Epoch 2/10, Average Loss: 0.35046567689735914
Epoch 3/10, Average Loss: 0.3504493251008895
Epoch 4/10, Average Loss: 0.3504767712892838
Epoch 5/10, Average Loss: 0.3504932030167395
Epoch 6/10, Average Loss: 0.35050492353231005
Epoch 7/10, Average Loss: 0.35051325668119687
Epoch 8/10, Average Loss: 0.35050456835662275
Epoch 9/10, Average Loss: 0.35046399449115817
Epoch 10/10, Average Loss: 0.35040467370626993
Epoch 1/15, Average Loss: 4.014158828241413
Epoch 2/15, Average Loss: 0.35046567689735914
Epoch 3/15, Average Loss: 0.3504493251008895
Epoch 4/15, Average Loss: 0.3504767712892838
Epoch 5/15, Average Loss: 0.3504932030167395
Epoch 6/15, Average Loss: 0.35050492353231005
Epoch 7/15, Average Loss: 0.3505132566811968

In [28]:
grid_results_df

,Hidden Layer Sizes,Activation Function,Learning Rate,Batch Size,Number of Epochs,Accuracy (Train),Accuracy (Test),Training Time,dataset
0,"[25, 30]",tanh,0.010,32,10,0.441324,0.447059,3.507027,wine_quality
1,"[25, 30]",tanh,0.010,32,15,0.443247,0.462745,3.650189,wine_quality
2,"[25, 30]",tanh,0.010,64,10,0.441324,0.447059,2.085237,wine_quality
3,"[25, 30]",tanh,0.010,64,15,0.443247,0.462745,2.656608,wine_quality
4,"[25, 30]",tanh,0.001,32,10,0.449211,0.453595,1.719398,wine_quality
...,...,...,...,...,...,...,...,...,...
91,"[20, 25, 30]",relu,0.010,64,15,0.888407,0.883103,16.645287,bank_marketing
92,"[20, 25, 30]",relu,0.001,32,10,0.898695,0.892814,11.018878,bank_marketing
93,"[20, 25, 30]",relu,0.001,32,15,0.898695,0.892814,17.024868,bank_marketing
94,"[20, 25, 30]",relu,0.001,64,10,0.898695,0.892814,10.937377,bank_marketing


In [29]:
# Test Random Search over all three datasets

In [38]:
num_iterations = 10
param_ranges = {
    'min_hidden': 5,
    'max_hidden': 50,
    'min_layers': 1,
    'max_layers': 3,
    'activation_functions': [F.relu, F.tanh, F.sigmoid],
    'min_lr': 0.001,
    'max_lr': 0.1,
    'batch_sizes': [32, 64, 128],
    'num_epochs': [5, 10, 15]
}

initial_configuration = {
    'Hidden Layer Sizes': [25],
    'Activation Function': F.relu,
    'Learning Rate': 0.01,
    'Batch Size': 64,
    'Number of Epochs': 10
}



all_random_results = []

for dataset_name, dataset in datasets.items():
    train_X, train_Y, test_X, test_Y = train_test_split(dataset, "class", return_torch=True)

    train_data = TensorDataset(train_X, train_Y)
    train_loader = DataLoader(train_data, batch_size=32, shuffle=False)

    test_data = TensorDataset(test_X, test_Y)
    test_loader = DataLoader(test_data, batch_size=32, shuffle=False)
    
    input_size = train_X.shape[1]

    if dataset_name == 'wine_quality':
        num_classes = 10
    else:
        num_classes = len(np.unique(train_Y))

    best_config, best_accuracy, random_results = random_search(num_iterations, initial_configuration, param_ranges, train_loader, test_loader)

    random_results['dataset'] = dataset_name
    all_random_results.append(random_results)
#all_random_results = pd.DataFrame(all_random_results)
#random_results_df = pd.concat(all_random_results, ignore_index=True)


Epoch 1/10, Average Loss: 1.3060762721336692
Epoch 2/10, Average Loss: 1.277217772109377
Epoch 3/10, Average Loss: 1.2667854095529194
Epoch 4/10, Average Loss: 1.2539941251643596
Epoch 5/10, Average Loss: 1.2450703005849217
Epoch 6/10, Average Loss: 1.2147458192029614
Epoch 7/10, Average Loss: 1.1898289940839897
Epoch 8/10, Average Loss: 1.1940654428458652
Epoch 9/10, Average Loss: 1.1851941292271293
Epoch 10/10, Average Loss: 1.1793131930696452
Epoch 1/15, Average Loss: 1.3429917122688761
Epoch 2/15, Average Loss: 1.3290214728724006
Epoch 3/15, Average Loss: 1.3277870789627355
Epoch 4/15, Average Loss: 1.3282437075866511
Epoch 5/15, Average Loss: 1.328351981800758
Epoch 6/15, Average Loss: 1.3280822111785047
Epoch 7/15, Average Loss: 1.328296303017739
Epoch 8/15, Average Loss: 1.328261375427246
Epoch 9/15, Average Loss: 1.3282252469677136
Epoch 10/15, Average Loss: 1.3282340114102043
Epoch 11/15, Average Loss: 1.3282237623366842
Epoch 12/15, Average Loss: 1.328217453020482
Epoch 13/15

Epoch 15/15, Average Loss: 0.31103335569302243
Epoch 1/15, Average Loss: 0.8347045580546061
Epoch 2/15, Average Loss: 0.6755234003067017
Epoch 3/15, Average Loss: 0.5609065989653269
Epoch 4/15, Average Loss: 0.5300098309914271
Epoch 5/15, Average Loss: 0.5074907938639323
Epoch 6/15, Average Loss: 0.4191334918141365
Epoch 7/15, Average Loss: 0.45527692635854083
Epoch 8/15, Average Loss: 0.3510527138908704
Epoch 9/15, Average Loss: 0.33509618292252225
Epoch 10/15, Average Loss: 0.3505646089712779
Epoch 11/15, Average Loss: 0.29779554158449173
Epoch 12/15, Average Loss: 0.2730310186743736
Epoch 13/15, Average Loss: 0.2706012614071369
Epoch 14/15, Average Loss: 0.3459014830489953
Epoch 15/15, Average Loss: 0.21344163765509924
Epoch 1/15, Average Loss: 0.6707229812939962
Epoch 2/15, Average Loss: 0.6667693456013998
Epoch 3/15, Average Loss: 0.6637280484040579
Epoch 4/15, Average Loss: 0.6531555751959482
Epoch 5/15, Average Loss: 0.6374999582767487
Epoch 6/15, Average Loss: 0.607203066349029

In [39]:
random_results_df = pd.DataFrame(all_random_results)
random_results_df = pd.concat(all_random_results, ignore_index=True)

In [40]:
random_results_df

,Hidden Layer Sizes,Activation Function,Learning Rate,Batch Size,Number of Epochs,Accuracy,Training Time,dataset
0,"[12, 38]",sigmoid,0.013719,128,10,0.456209,2.065076,wine_quality
1,"[32, 14, 10]",tanh,0.073446,32,15,0.284967,2.933734,wine_quality
2,"[14, 15, 6]",relu,0.038363,64,5,0.458824,0.871133,wine_quality
3,"[36, 17, 24]",tanh,0.075953,32,10,0.284967,1.847184,wine_quality
4,"[27, 36]",tanh,0.036705,32,5,0.284967,0.806684,wine_quality
5,"[17, 20, 11]",relu,0.053805,128,15,0.458824,2.676784,wine_quality
6,"[34, 43]",relu,0.002514,64,10,0.397386,1.622344,wine_quality
7,"[5, 20, 27]",relu,0.046287,32,10,0.458824,1.786368,wine_quality
8,[33],relu,0.035976,128,10,0.418301,1.354204,wine_quality
9,"[34, 38, 13]",tanh,0.085812,32,5,0.284967,0.880774,wine_quality
